#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

'unzip' is not recognized as an internal or external command,
operable program or batch file.


# Dataset Loading and Pre-Processing

In [4]:
import pandas as pd
data = pd.read_csv("./content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [8]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
10111,3.0,88,5.0,54.0,overcast,987.78,0,Winter,3.0,mountain,Snowy
12563,33.0,94,13.5,67.0,overcast,997.77,3,Spring,4.5,coastal,Rainy
9889,-2.0,92,9.0,66.0,overcast,998.83,1,Winter,3.5,mountain,Snowy
7845,30.0,47,8.5,13.0,partly cloudy,1020.78,5,Spring,6.0,inland,Sunny
10622,16.0,97,19.0,59.0,overcast,1005.12,3,Summer,1.0,inland,Rainy


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [25]:
# START CODE HERE
object_columns = list(data.select_dtypes(include=['object']).columns)
non_object_columns = list(data.select_dtypes(exclude=['object']).columns)
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [10]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [11]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

column_transformer = ColumnTransformer([('encoder', OrdinalEncoder(), object_columns),('scaler', StandardScaler(), non_object_columns)])

data_scaled_labeled = column_transformer.fit_transform(data)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)
encoder = column_transformer.named_transformers_['encoder']
weather_type_mapping = encoder.categories_[0]


all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

**TEST** (DON'T EDIT THIS CELL)

In [14]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("./content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [15]:
X = data_scaled_labeled.drop('Weather Type',axis=1)
y = data_scaled_labeled['Weather Type']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.9, random_state = 10)

**TEST** (DON'T EDIT THIS CELL)

In [16]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [17]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,2.0,3.0,2.0,-0.985155,0.212404,-0.554712,0.136346,-0.551315,-1.038715,-0.878846
12278,3.0,0.0,2.0,0.625367,-1.966517,-0.192836,-1.147097,0.132859,1.295035,0.604230
2859,2.0,3.0,1.0,-0.064857,0.955217,0.820417,1.075450,-0.220922,-0.779410,-0.582231
5495,3.0,1.0,2.0,0.625367,0.360966,0.675666,-0.896669,0.252488,-0.520104,0.900845
8469,0.0,0.0,1.0,0.740404,-1.223703,-0.699463,-1.428828,0.613527,1.295035,0.010999


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

In [30]:
from tensorflow import keras

model = keras.Sequential([keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),keras.layers.Dense(32, activation='relu'),keras.layers.Dense(4, activation='softmax')])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=15)
loss, score = model.evaluate(X_test,y_test)

print("Accuracy:", score*100)

Epoch 1/15


C:\Users\pravm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


372/372 ━━━━━━━━━━━━━━━━━━━━ 1s 958us/step - accuracy: 0.7431 - loss: 0.8023
Epoch 2/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - accuracy: 0.8938 - loss: 0.3377
Epoch 3/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - accuracy: 0.9034 - loss: 0.2793
Epoch 4/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step - accuracy: 0.9003 - loss: 0.2681
Epoch 5/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - accuracy: 0.9052 - loss: 0.2500
Epoch 6/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step - accuracy: 0.9078 - loss: 0.2350
Epoch 7/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step - accuracy: 0.9066 - loss: 0.2360
Epoch 8/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - accuracy: 0.9105 - loss: 0.2204
Epoch 9/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - accuracy: 0.9139 - loss: 0.2048
Epoch 10/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - accuracy: 0.9181 - loss: 0.1984
Epoch 11/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - accuracy: 0.9216 - loss: 0.1870
Epoch 12/15
372/372 ━━━━━━━━━━━━━━━━━━━━ 

**TEST** (DON'T EDIT THIS CELL)

In [29]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [22]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,0.0,2.0,0.0,1.027998,-1.619871,-0.627087,-1.178401,0.439325,0.517118,0.010999
2331,3.0,3.0,2.0,-0.985155,-1.372266,0.675666,1.263271,3.815297,1.813646,-1.323769
217,2.0,3.0,1.0,-1.560341,1.351385,0.820417,-0.051475,-0.384102,-1.038715,-0.285616
1760,2.0,0.0,0.0,0.682886,0.360966,0.024290,0.042435,-0.038655,-0.520104,-0.137308
3963,2.0,3.0,1.0,0.740404,-0.084722,-0.337587,-0.427117,0.287974,-0.520104,0.010999


In [23]:
y = model.predict(input)
y

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


array([[3.3599880e-04, 6.6417168e-05, 1.2398976e-03, 9.9835771e-01],
       [1.9779232e-01, 1.2688161e-01, 5.5293572e-01, 1.2239035e-01],
       [9.8598430e-05, 5.9366110e-05, 9.9971455e-01, 1.2747690e-04],
       [3.0695322e-01, 6.8923759e-01, 2.1151800e-04, 3.5976570e-03],
       [9.6976054e-01, 2.6970977e-02, 2.2101629e-04, 3.0474386e-03]],
      dtype=float32)

Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.


In [24]:
pred = weather_type_mapping[np.argmax(y, axis=1)]
pred

array(['partly cloudy', 'overcast', 'overcast', 'cloudy', 'clear'],
      dtype=object)